# Imports

In [ ]:
import json
import locale
import os
from datetime import datetime

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from stock_modules.stock_transform import (create_batch_xy,
                                           create_transformer_onehot_xy)

from stock_modules.stock_ml import (create_transformer_model,
                                    MultiSoftmaxLoss, MultiAccuracy)

import tensorflow as tf
from tensorflow import keras
from keras.utils import plot_model

# Constants

In [ ]:
ENCODING = locale.getpreferredencoding()
DF_PATH = "HEL_12-10-21to08-11-23.csv"
HISTORY_ARRAY_PATH = "./histories_arr.npy"
MODEL_PATH = "./model.h5"
SELECTED_TICKERS_PATH = "./TICKERS_TO_FOLLOW.json"

TEST_FRAC = 0.2
PREDICT_PRICES = False

# Data Import

In [ ]:
SELECTED_TICKERS = json.load(open(SELECTED_TICKERS_PATH,
                                  "r", encoding=ENCODING))
DATAFRAME = pd.read_csv(DF_PATH, encoding=ENCODING)

DATAFRAME.set_index("date", inplace=True)
HAS_TIMEDELTA = "Time Delta" in DATAFRAME.columns

# ind transformation tells the label of each index in the np_arr_test
IND_CONVERSION = {i: ticker for i, ticker in enumerate(DATAFRAME.columns) if ticker in SELECTED_TICKERS}
IND_CONVERSION = {i: ticker for i, ticker in enumerate(IND_CONVERSION.values())}

print("Selected tickers: \n", SELECTED_TICKERS)
print("Dataframe columns: \n", DATAFRAME.columns)
print("Dataframe shape: ", DATAFRAME.shape)
print("Dataframe head: \n", DATAFRAME.head(2))
print(f"Index conversion: \n {IND_CONVERSION}")

# Data Treatment

In [ ]:
test_begin_idx = int(DATAFRAME.shape[0] * (1 - TEST_FRAC))

if PREDICT_PRICES:
    scaler = MinMaxScaler()

    scaler.fit(DATAFRAME.iloc[:test_begin_idx, :])
    transformed_df = pd.DataFrame(scaler.transform(DATAFRAME), columns=DATAFRAME.columns, index=DATAFRAME.index)
    transformed_np_arr = transformed_df.to_numpy()

    def inverse_transform(df):
        if isinstance(df, pd.DataFrame):
            return pd.DataFrame(scaler.inverse_transform(df), columns=df.columns, index=df.index)
        elif isinstance(df, np.ndarray):
            return scaler.inverse_transform(df)

# If we are predicting the up/down, we create a dataframe where we subtract the previous value from the current value
else:
    # Do not diff the Time Delta column
    df = DATAFRAME.copy()
    if HAS_TIMEDELTA:
        td_col = df["Time Delta"]
        df.drop("Time Delta", axis=1, inplace=True)
    transformed_df = df.diff()
    # The first row is NaN, so lets copy the second row there
    transformed_df.iloc[0, :] = transformed_df.iloc[1, :]
    # Add back the Time Delta column
    if HAS_TIMEDELTA:
        transformed_df["Time Delta"] = td_col
        # Make Time Delta the first column
        cols = transformed_df.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        transformed_df = transformed_df[cols]
    transformed_np_arr = transformed_df.to_numpy()

    def inverse_transform(df):
        return df

print("Transformed df: \n", transformed_df.head(2))
print("Transformed df shape: ", transformed_df.shape)

# Model Optimization

In [ ]:
if not os.path.exists("./models/"):
    os.makedirs("./models/")

METRICS = [MultiAccuracy()]
CALLBACKS = [keras.callbacks.EarlyStopping(patience=10,
                                           restore_best_weights=True)]
CLASS_FIRST = True

counter = 0
RESUME = 0

for memory_length in [10,50,200]:
    if PREDICT_PRICES:
        OUTPUT_SCALE = (0,1)
        x, y = create_batch_xy(
                    memory_length,
                    transformed_np_arr,
                    overlap=True,
                    y_updown=False,
                    diff_data=True,
                    output_scale=OUTPUT_SCALE)
    else:
        x, x_ts, y = create_transformer_onehot_xy(
                            memory_length,
                            transformed_np_arr,
                            DATAFRAME.to_numpy(),
                            DATAFRAME.index.to_numpy(),
                            0.01)

    split_idx = int(x.shape[0] * (1 - TEST_FRAC))

    x_train = x[:split_idx,:,:]
    x_ts_train = x_ts[:split_idx,:,:]
    y_train = y[:split_idx,:,:]

    x_test = x[split_idx:,:,:]
    x_ts_test = x_ts[split_idx:,:,:]
    y_test = y[split_idx:,:,:]

    if CLASS_FIRST:
        y_train = tf.transpose(y_train, (0,2,1))
        y_test = tf.transpose(y_test, (0,2,1))

    for head_size in [32,16,8]:
        for num_heads in [32,16,8]:
            for ff_dim in [64,32,16]:
                for num_transformer_blocks in [4,2,1]:
                    for mlp_units in [64,32,16]:
                        if counter < RESUME:
                            counter = counter + 1
                            continue
                        
                        serial = "transformer_model_" \
                            + datetime.now().strftime("%Y%m%d%H%M%S")
                        model_dict = {
                            "serial": serial,
                            "memory_length": memory_length,
                            "head_size": head_size,
                            "num_heads": num_heads,
                            "ff_dim": ff_dim,
                            "num_transformer_blocks": num_transformer_blocks,
                            "mlp_units": mlp_units
                        }

                        model = create_transformer_model(
                                m=memory_length+1,
                                n=len(SELECTED_TICKERS),
                                output_dim=3,
                                head_size=head_size,
                                num_heads=num_heads,
                                ff_dim=ff_dim,
                                num_transformer_blocks=num_transformer_blocks,
                                mlp_units=(mlp_units,),
                                class_first=CLASS_FIRST
                            )
                        

                        model.compile(optimizer=keras.optimizers.Adam(),
                                    loss=MultiSoftmaxLoss(),
                                    metrics=METRICS)

                        model.fit(x = (x_train,x_ts_train,x_train,x_ts_train),
                                y = y_train,
                                batch_size=32,
                                epochs=100,
                                validation_split=0.25,
                                callbacks=CALLBACKS)
                        
                        model.save("./models/"+serial+".keras")

                        model_dict.update(model.evaluate(
                                    x = (x_test,x_ts_test,x_test,x_ts_test),
                                    y = y_test,
                                    batch_size=32,
                                    workers=4,
                                    use_multiprocessing=True,
                                    return_dict=True
                                )
                            )
                        
                        if os.path.exists("./transformer_results.json"):
                            with open("./transformer_results.json", "r",
                                      encoding=ENCODING) as json_file:
                                model_list = json.load(json_file)
                        else:
                            model_list = [model_dict]
                        
                        with open("./transformer_results.json", "w",
                                  encoding=ENCODING) as json_file:
                            json.dump(model_list, json_file)

plot_model(
        model,
        to_file="./figures/transformer_model_plot.png",
        show_shapes=True,
        show_layer_names = True
    )